# Listas de Acuerdos - Diario

Elaborar mensaje para enviar vía correo electrónico cada día.

In [ ]:
from datetime import date

# Pandas
import pandas as pd

# SQLAlchemy
from plataforma_web.app import create_app
from plataforma_web.extensions import db
app = create_app()
db.app = app

# Plataforma Web
from plataforma_web.blueprints.distritos.models import Distrito
from plataforma_web.blueprints.autoridades.models import Autoridad
from plataforma_web.blueprints.listas_de_acuerdos.models import ListaDeAcuerdo

In [ ]:
# Distritos
distritos_select = db.session.query(Distrito.id, Distrito.nombre).\
    filter(Distrito.es_distrito_judicial == True).\
    filter(Distrito.estatus == "A").statement
distritos = pd.read_sql_query(sql=distritos_select, con=db.engine)
for index, row in distritos.iterrows():
    print(row['id'], row['nombre'])

In [ ]:
# Distrito Judicial
distrito = Distrito.query.get(6)  # 6: Saltillo, 8: Torreon, 14: Salas

# Juzgados
autoridades_select = db.session.query(Autoridad.id, Autoridad.clave, Autoridad.descripcion_corta).\
    filter(Autoridad.distrito_id == distrito.id).\
    filter(Autoridad.es_jurisdiccional == True).\
    filter(Autoridad.es_notaria == False).\
    filter(Autoridad.estatus == "A").\
    order_by(Autoridad.clave).statement
autoridades = pd.read_sql_query(sql=autoridades_select, con=db.engine)
autoridades

In [ ]:
# Fecha
fecha = date(year=2021, month=9, day=2)

# Listas de acuerdos de esa fecha
listas_de_acuerdos_select = db.session.query(Autoridad.clave, ListaDeAcuerdo.archivo, ListaDeAcuerdo.creado).\
    join(Autoridad).\
    filter(ListaDeAcuerdo.autoridad_id.in_(autoridades.id)).\
    filter(ListaDeAcuerdo.fecha == fecha).\
    filter(ListaDeAcuerdo.estatus == "A").\
    order_by(ListaDeAcuerdo.creado).statement
listas_de_acuerdos = pd.read_sql_query(sql=listas_de_acuerdos_select, con=db.engine)
listas_de_acuerdos.columns = ['clave2', 'archivo', 'creado']

# Reporte diario
diario = pd.merge(autoridades, listas_de_acuerdos, left_on='clave', right_on='clave2', how='left').drop('clave2', axis=1)
diario